# Der erwartungswert im allgemeinen Fall mit Hilfe des Gleichungssystem

In [1]:
import numpy as np
import pandas as pd 
import copy
import itertools as it
from itertools import product
import scipy
from scipy.special import comb
import pprint
import sympy
import math

## Erzeugen der Dyck-Wörter mit Hilfe der Positionen von der '-1' 

In [2]:
def Positions_Minuses_inDyckwords(W):
    #gernrating the dyck words with the length (2k)
    k =int(sum(W)/2)
    #the array of the positions of the 1's (the "-")
    POS=np.zeros([1,k], dtype = int)
    #the number of the perfect matching of the set {1,2k}, which ist equivalent to each Dyck Word
    Coeff=np.ones([0,1], int)
    A=np.zeros([1,k], dtype = int)
    #the Dyck words represented mit "+","-"
    Dyck_Words=np.full([0,2*k], +1)
    #the first( smallest ) word
    for i in range(0,k):
      POS[0][i]=2*(i+1)
      A[0][i]=2*(i+1)
    ind=True
    while ind == True:
        ind=False
        C=np.full([1,2*k], +1)
        B=np.ones([1,1], dtype = int)
        for i in range(0,k):
            B=B*(A[0][i]+1-2*(i+1))
            C[0][A[0][i]-1]=-1
        Coeff=np.concatenate((Coeff,B),axis=0)
        Dyck_Words = np.concatenate((Dyck_Words,C),axis=0)
        for i in range(k-2,-1,-1):
            if A[0][i]< k+1+i:
               A[0][i]=A[0][i]+1
               for j  in range(i+1,k-1):
                   A[0][j]=max(A[0][j-1]+1,2*(j+1))
               ind=True
               POS=np.concatenate((POS,A), axis = 0)
               break
    return(Dyck_Words)

In [3]:
def find_the_Projektions(W):
    POS=Positions_Minuses_inDyckwords(W)
    indexes=W[:-1]
    return(np.unique(np.add.reduceat(POS,np.cumsum([0,*indexes]),axis=1),axis=0))

In [4]:
def rounded_halves(A):
    X=[]
    for i in range(len(A)):
        X.append(list(range(int(np.floor(A[i]/2)+1))))
    return(X)
def Possible_Graphs(W):
    Graphs=np.zeros((0,len(W)),dtype=int)
    X=rounded_halves(W)
    cartesian_product = [i for i in it.product(*X)]
    for x in cartesian_product:
          A=np.array(x,int)
          Graph=np.array(2*A+(W%2),int)
          Diff=sum(Graph)-2*Graph[np.argmax(Graph)]
          if Diff>=0:
                Graphs=np.concatenate((Graphs,Graph.reshape(1,len(W))),axis=0)
    return(Graphs)

In [5]:
def Projektions_and_its_Graphs(W):
    Graphs=Possible_Graphs(W)
    Projektions=find_the_Projektions(W)
    Graphs_of_a_Projektion=[]
    for x in Projektions:
        absolute=np.absolute(x)
        graphs_of_x=np.zeros((0,len(x)),dtype=int)
        for y in Graphs:
            if absolute[0]==y[0] and absolute[-1]==y[-1] and (absolute<=y).all():
                graphs_of_x=np.concatenate((graphs_of_x,y.reshape(1,len(y))),axis=0)
        Graphs_of_a_Projektion.append([x,graphs_of_x])
    return(Graphs_of_a_Projektion)

In [6]:
def Matrix_of_the_leqs_(k):
    L=list(it.combinations(range(1,k+1),2))
    M=np.zeros((2*k,len(L)),int)
    for i in range(1,k+1):
        for j in range(len(L)):
            if i == L[j][0]:
                M[i-1,j]=1
                M[k+i-1,j]=1
            elif i==L[j][1]:
                M[i-1,j]=1
                M[k+i-1,j]=-1
    return(M)

In [22]:
W=np.array([1,2,3,4])
Projektions_and_its_Graphs1=Projektions_and_its_Graphs(W)

In [23]:
Matrix=Matrix_of_the_leqs_(len(W))

In [24]:
Matrix

array([[ 1,  1,  1,  0,  0,  0],
       [ 1,  0,  0,  1,  1,  0],
       [ 0,  1,  0,  1,  0,  1],
       [ 0,  0,  1,  0,  1,  1],
       [ 1,  1,  1,  0,  0,  0],
       [-1,  0,  0,  1,  1,  0],
       [ 0, -1,  0, -1,  0,  1],
       [ 0,  0, -1,  0, -1, -1]])

In [41]:
Rightside_Equqtion_system=[]
for x in Projektions_and_its_Graphs1:
    v=[]
    for y in x[1]:
        v.append(np.concatenate((y,x[0])))
    Rightside_Equqtion_system.append([x[0],v])

In [42]:
from sympy import Matrix, symbols, linsolve

In [43]:
linsolve(Matrix,Rightside_Equqtion_system[0][1][0])

ValueError: Invalid arguments

In [33]:
Equqtion_system[0][1][0]

array([ 1,  0,  1,  0,  1,  0, -1,  0])

In [39]:
Rightside_Equqtion_system

[]